In [2]:
import pandas as pd

path = "/Users/hwangjeonghyun/Downloads/Covid Data.csv"
df = pd.read_csv(path)

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', None)      
pd.set_option('display.max_colwidth', None)

In [4]:
df.head()

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,COPD,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,03/05/2020,97,1,65,2,2,2,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,03/06/2020,97,1,72,97,2,2,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,09/06/2020,1,2,55,97,1,2,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,12/06/2020,97,2,53,2,2,2,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,21/06/2020,97,2,68,97,1,2,2,2,1,2,2,2,2,2,3,97


In [5]:
df.shape

(1048575, 21)

In [6]:
# 숫자형
num_cols = df.select_dtypes(include='number').columns.tolist()

# 문자형 (object + category)
cat_cols = df.select_dtypes(include=['object','category']).columns.tolist()

print("숫자형:", num_cols)
print("문자형:", cat_cols)


숫자형: ['USMER', 'MEDICAL_UNIT', 'SEX', 'PATIENT_TYPE', 'INTUBED', 'PNEUMONIA', 'AGE', 'PREGNANT', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR', 'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY', 'RENAL_CHRONIC', 'TOBACCO', 'CLASIFFICATION_FINAL', 'ICU']
문자형: ['DATE_DIED']


In [7]:
missing_info = df.isnull().sum().to_frame(name='MissingCount')
missing_info['MissingRatio'] = (missing_info['MissingCount'] / len(df)).round(4)

print(missing_info.sort_values('MissingRatio', ascending=False))


                      MissingCount  MissingRatio
USMER                            0           0.0
ASTHMA                           0           0.0
CLASIFFICATION_FINAL             0           0.0
TOBACCO                          0           0.0
RENAL_CHRONIC                    0           0.0
OBESITY                          0           0.0
CARDIOVASCULAR                   0           0.0
OTHER_DISEASE                    0           0.0
HIPERTENSION                     0           0.0
INMSUPR                          0           0.0
COPD                             0           0.0
MEDICAL_UNIT                     0           0.0
DIABETES                         0           0.0
PREGNANT                         0           0.0
AGE                              0           0.0
PNEUMONIA                        0           0.0
INTUBED                          0           0.0
DATE_DIED                        0           0.0
PATIENT_TYPE                     0           0.0
SEX                 

In [8]:
import numpy as np

# 질환 관련 컬럼들
disease_cols = [
    'PNEUMONIA','DIABETES','COPD','ASTHMA','INMSUPR',
    'HIPERTENSION','OTHER_DISEASE','CARDIOVASCULAR',
    'OBESITY','RENAL_CHRONIC','TOBACCO'
]

# 98을 'Unknown'으로 치환
for col in disease_cols:
    df[col] = df[col].replace(98, 'Unknown')

# 확인
for col in disease_cols:
    print(col, df[col].value_counts())


PNEUMONIA PNEUMONIA
2     892534
1     140038
99     16003
Name: count, dtype: int64
DIABETES DIABETES
2          920248
1          124989
Unknown      3338
Name: count, dtype: int64
COPD COPD
2          1030510
1            15062
Unknown       3003
Name: count, dtype: int64
ASTHMA ASTHMA
2          1014024
1            31572
Unknown       2979
Name: count, dtype: int64
INMSUPR INMSUPR
2          1031001
1            14170
Unknown       3404
Name: count, dtype: int64
HIPERTENSION HIPERTENSION
2          882742
1          162729
Unknown      3104
Name: count, dtype: int64
OTHER_DISEASE OTHER_DISEASE
2          1015490
1            28040
Unknown       5045
Name: count, dtype: int64
CARDIOVASCULAR CARDIOVASCULAR
2          1024730
1            20769
Unknown       3076
Name: count, dtype: int64
OBESITY OBESITY
2          885727
1          159816
Unknown      3032
Name: count, dtype: int64
RENAL_CHRONIC RENAL_CHRONIC
2          1026665
1            18904
Unknown       3006
Name: count, dtyp

In [9]:
# DATE_DIED 기반으로 DEATH 컬럼 생성
# 0 = Alive, 1 = Death
df["DEATH"] = df["DATE_DIED"].apply(lambda x: 0 if x=="9999-99-99" else 1)

# 확인
print(df["DEATH"].value_counts())


DEATH
0    971633
1     76942
Name: count, dtype: int64


In [10]:
import pandas as pd

def clean_pregnant(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out['PREGNANT_CLEAN'] = pd.Series(index=out.index, dtype='object')

    # 남성: 임신 개념 미적용 → 모두 NotApplicable
    mask_male = (out['SEX'] == 2)
    out.loc[mask_male, 'PREGNANT_CLEAN'] = 'NotApplicable'

    # 여성: 코드 매핑
    mask_female = (out['SEX'] == 1)
    mapping_female = {
        1: 'Yes',         # 임신
        2: 'No',          # 임신 아님
        97: 'Unknown',    # 정보 없음
        98: 'Unknown',    # 적용 불가(실무상 에러로 간주 → Unknown)
        99: 'Unknown'     # 무응답
    }
    out.loc[mask_female, 'PREGNANT_CLEAN'] = (
        out.loc[mask_female, 'PREGNANT']
           .map(mapping_female)
           .fillna('Unknown')
    )

    # 카테고리형으로 정리(원-핫은 모델링 직전에)
    out['PREGNANT_CLEAN'] = pd.Categorical(
        out['PREGNANT_CLEAN'],
        categories=['NotApplicable','No','Yes','Unknown'],
        ordered=False
    )
    return out

# 사용
df = clean_pregnant(df)

# 빠른 점검
print(pd.crosstab(df['SEX'], df['PREGNANT_CLEAN']))
print(df['PREGNANT_CLEAN'].value_counts(dropna=False))


PREGNANT_CLEAN  NotApplicable      No   Yes  Unknown
SEX                                                 
1                           0  513179  8131     3754
2                      523511       0     0        0
PREGNANT_CLEAN
NotApplicable    523511
No               513179
Yes                8131
Unknown            3754
Name: count, dtype: int64


In [11]:
def clean_intubed(df):
    out = df.copy()
    mapping = {
        1: 'Yes',
        2: 'No',
        97: 'Unknown',
        99: 'Unknown'
    }
    out['INTUBED_CLEAN'] = out['INTUBED'].map(mapping).fillna('Unknown')
    out['INTUBED_CLEAN'] = pd.Categorical(out['INTUBED_CLEAN'],
                                          categories=['No','Yes','Unknown'],
                                          ordered=False)
    return out

# 적용
df = clean_intubed(df)

# 분포 확인
print(df['INTUBED_CLEAN'].value_counts())


INTUBED_CLEAN
Unknown    855869
No         159050
Yes         33656
Name: count, dtype: int64


In [12]:
def clean_icu(df):
    out = df.copy()
    mapping = {
        1: 'Yes',
        2: 'No',
        97: 'Unknown',
        99: 'Unknown'
    }
    out['ICU_CLEAN'] = out['ICU'].map(mapping).fillna('Unknown')
    out['ICU_CLEAN'] = pd.Categorical(out['ICU_CLEAN'],
                                      categories=['No','Yes','Unknown'],
                                      ordered=False)
    return out

# 적용
df = clean_icu(df)

# 확인
print(df['ICU_CLEAN'].value_counts())


ICU_CLEAN
Unknown    856032
No         175685
Yes         16858
Name: count, dtype: int64


In [13]:
# 원본 컬럼 삭제 (이미 전처리된 버전 있음)
df.drop(columns=["INTUBED","ICU","DATE_DIED", "PREGNANT"], inplace=True)

# 확인
print("현재 컬럼 수:", df.shape[1])
print("남아있는 컬럼:", df.columns.tolist())


현재 컬럼 수: 21
남아있는 컬럼: ['USMER', 'MEDICAL_UNIT', 'SEX', 'PATIENT_TYPE', 'PNEUMONIA', 'AGE', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR', 'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY', 'RENAL_CHRONIC', 'TOBACCO', 'CLASIFFICATION_FINAL', 'DEATH', 'PREGNANT_CLEAN', 'INTUBED_CLEAN', 'ICU_CLEAN']


In [14]:
# 기관 코드 삭제 
df.drop(columns=["USMER","MEDICAL_UNIT"], inplace=True)

# 확인
print("현재 컬럼 수:", df.shape[1])
print("남아있는 컬럼:", df.columns.tolist())


현재 컬럼 수: 19
남아있는 컬럼: ['SEX', 'PATIENT_TYPE', 'PNEUMONIA', 'AGE', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR', 'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY', 'RENAL_CHRONIC', 'TOBACCO', 'CLASIFFICATION_FINAL', 'DEATH', 'PREGNANT_CLEAN', 'INTUBED_CLEAN', 'ICU_CLEAN']


In [ ]:
# 삭제할 컬럼 (내생변수)
drop_cols = ["CLASIFFICATION_FINAL", "INTUBED_CLEAN", "ICU_CLEAN", "PATIENT_TYPE"]

df.drop(columns=drop_cols, inplace=True)

print("최종 컬럼:", df.columns.tolist())


최종 컬럼: ['SEX', 'PNEUMONIA', 'AGE', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR', 'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY', 'RENAL_CHRONIC', 'TOBACCO', 'DEATH', 'PREGNANT_CLEAN']


In [16]:
# 전처리된 데이터 확인
df.head()

,SEX,PNEUMONIA,AGE,DIABETES,COPD,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,DEATH,PREGNANT_CLEAN
0,1,1,65,2,2,2,2,1,2,2,2,2,2,1,No
1,2,1,72,2,2,2,2,1,2,2,1,1,2,1,NotApplicable
2,2,2,55,1,2,2,2,2,2,2,2,2,2,1,NotApplicable
3,1,2,53,2,2,2,2,2,2,2,2,2,2,1,No
4,2,2,68,1,2,2,2,1,2,2,2,2,2,1,NotApplicable


In [17]:
# 현재 df를 CSV로 저장
df.to_csv("covid_preprocessed.csv", index=False, encoding="utf-8")

print("CSV 저장 완료 -> covid_preprocessed.csv")


CSV 저장 완료 -> covid_preprocessed.csv
